In [1]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset

In [2]:
df = pd.read_csv("dataset/dataset-checkpoint-1.csv", encoding='ISO-8859-1', engine='python')
df.head(1000000)

,Id,comment,quality,sentiment,positive-attribute,negative-attribute
0,45881,"Rand was a good guy, really... he was. But I f...",awful,0,NaN,unfair
1,45881,"He is brilliant in his field, but expects his ...",poor,0,NaN,unfair
2,45881,He has a major in philosophy which is why all ...,awful,0,NaN,unfair
3,45881,I thought his class was extremely hard but des...,good,1,NaN,NaN
4,45881,I recently just finished taking Comp 110 with ...,poor,0,NaN,unfair
...,...,...,...,...,...,...
2271,64144,Such a bad bad class.,poor,0,NaN,boring
2272,64144,He was a funny guy but a horendous teacher. Al...,poor,0,NaN,unhelpful
2273,64144,Prof Graves is a very interesting man...if you...,average,1,NaN,NaN
2274,64144,"He's really disoriented, but a really cool guy...",average,1,NaN,NaN


In [3]:
# New synthetic dataset
synth_df = pd.DataFrame(columns=["comment","sentiment","positive-attribute","negative-attribute"])

In [4]:
test_prompt = (
    "<start_of_turn>user\n Please say the words 'Hello World' and nothing more. <end_of_turn>"
)

import ollama
response = ollama.chat(model='gemma2:27b', messages=[
  {
    'role': 'user',
    'content': test_prompt,
  },
])
print(response['message']['content'])


Hello World


In [5]:
# Define approved lists of positive and negative attributes
positive_attributes = [
    'knowledgeable',
    'engaging',
    'supportive',
    'clear',
    'passionate'
]

negative_attributes = [
    'confusing',
    'unfair',
    'boring',
    'unhelpful',
    'disorganized'
]

In [6]:
import random
# Prompt generator
def generate_data():
    sentiment = random.randint(0,1)
    if(sentiment == 0):
        sentiment_type = "negative"
        attribute = negative_attributes[random.randint(0,len(negative_attributes)-1)]
    else:
        sentiment_type = "positive"
        attribute = positive_attributes[random.randint(0,len(positive_attributes)-1)]
    
    return [sentiment,sentiment_type,attribute]

generate_data()

[0, 'negative', 'confusing']

In [7]:
def generate_comments(amount_comments, num_rows):
    comment_map = {}
    for i in range(amount_comments):
        comment_id = random.randint(0,num_rows)
        if comment_map.get(comment_id) is None:
            comment_map[comment_id] = df.loc[comment_id, 'comment']
        else:
            i-=1
    return comment_map
generate_comments(1,10)

{1: 'He is brilliant in his field, but expects his students to perform at high levels, with unrealistic expectations. Only by a miracle did I get an A, and that was after a heavy curve. He does care about his students though, as he gives lots of time in class for work, while he goes around giving tips and answering questions.'}

In [8]:
checkpoint = 0
for i in range(2000):
    full_prompt = f"<start_of_turn>user\n Here are a few examples of reviews students left for professors: \n "
    comments = generate_comments(12,2275)
    for key,value in comments.items():
        full_prompt += f"\n '{value}' \n"

    data = generate_data()
    sentiment = data[0]
    sentiment_type = data[1]
    attribute = data[2]
    
    full_prompt += f"\n You are playing the role of a student who is writing a review for their professor at the end of the semester. Please write a {sentiment_type} review for a professor who is {attribute}. Please refrain from using the word '{attribute}' in the review. Please output just this review and nothing else. Do not be too verbose, refrain from information that does not pertain to describing the professor as {attribute} in particular. Refrain from using generic words like 'overall' and try to follow the format of the provided examples as closely as possible." + " <end_of_turn>"
    response = ollama.chat(model='gemma2:27b', messages=[
      {
        'role': 'user',
        'content': full_prompt,
        'temperature':0.5
      },
    ])
    
    if(sentiment == 0):
        synth_df.loc[len(synth_df)] = [response['message']['content'], sentiment, None, attribute]
    else:
        synth_df.loc[len(synth_df)] = [response['message']['content'], sentiment, attribute, None]
    
    print(f"\n{response['message']['content']}+ \n attribute: {attribute}, sentiment: {sentiment_type}, count: {i+1}/2000 \n")
    if(i % 200 == 0):
        synth_df.to_csv(f'dataset/dataset2-checkpoints/dataset2-checkpoint-{checkpoint}.csv', index=False)
        checkpoint+=1
    


This class was incredibly frustrating. No matter how hard I tried, I felt lost and confused. It seemed like every time I asked a question in class, I got a vague answer that didn't really clarify anything.  I ended up teaching myself most of the material from the textbook. 


+ 
 attribute: unhelpful, sentiment: negative, count: 1/2000 


This class felt like it lasted forever.  Lectures were incredibly monotone and it was hard to stay focused. I found myself constantly checking the clock and wishing for it to be over.   
+ 
 attribute: boring, sentiment: negative, count: 2/2000 


Lectures felt very scattered and it was hard to tell what information was actually important.  The syllabus changed multiple times throughout the semester, which made it difficult to keep up with assignments and deadlines.  Sometimes we'd spend an entire class on a topic that wasn't even on the exam. 


+ 
 attribute: disorganized, sentiment: negative, count: 3/2000 


I felt lost most of the time in this c

In [9]:
synth_df.head(50)

,comment,sentiment,positive-attribute,negative-attribute
0,This class was incredibly frustrating. No matt...,0,None,unhelpful
1,This class felt like it lasted forever. Lectu...,0,None,boring
2,Lectures felt very scattered and it was hard t...,0,None,disorganized
3,I felt lost most of the time in this class. E...,0,None,confusing
4,This class was a real struggle. Professor Smit...,0,None,confusing
5,Professor [Professor's Last Name] clearly love...,1,passionate,None
6,This class felt impossible to succeed in. No ...,0,None,unhelpful
7,This class felt impossible to succeed in. No ...,0,None,unhelpful
8,This professor seems to have different expecta...,0,None,unfair
9,Professor Smith is fantastic! She truly cares ...,1,supportive,None


In [10]:
synth_df.to_csv('dataset/dataset-checkpoint-2-final.csv', index=False)